In [1]:
import torch
torch.cuda.is_available()

True

# 2.1.1. 入门

In [3]:
x = torch.arange(12)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [4]:
torch.zeros((2, 3, 4))
# 第一个维度0：里面有2个元素，以此类推。。。

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

# 2.1.2. 运算符
![数学表示法](imgs/2_1_2运算符1.png)

对于任意具有相同形状的张量， 常见的标准算术运算符（+、-、*、/和**）都可以被升级为按元素运算(elementwise)。
我们可以在**同一形状**的任意两个张量上调用按元素操作。


In [2]:
x = torch.tensor([1.0,2,4,8])
y = torch.tensor([2,2,2,2])

In [3]:
x + y, x - y, x*y, x/y

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]))

In [4]:
# “按元素”方式可以应用更多的计算，包括像求幂这样的一元运算符。
torch.exp(x)

tensor([2.7183e+00, 7.3891e+00, 5.4598e+01, 2.9810e+03])

除了按元素计算外，我们还可以执行线性代数运算，包括向量点积和矩阵乘法。 我们将在 2.3节中解释线性代数的重点内容

我们也可以把多个张量连结（concatenate）在一起， 把它们端对端地叠起来形成一个更大的张量。 我们只需要提供张量列表，并给出沿哪个轴连结。 下面的例子分别演示了当我们沿行（轴-0，形状的第一个元素） 和按列（轴-1，形状的第二个元素）连结两个矩阵时，会发生什么情况。

In [6]:
X = torch.arange(12,dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0,1,4,3],[1,2,3,4],[4,3,2,1]])
X, Y

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]),
 tensor([[2., 1., 4., 3.],
         [1., 2., 3., 4.],
         [4., 3., 2., 1.]]))

In [8]:
torch.cat((X,Y),dim=0), torch.cat((X,Y),dim=1)


(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

我们可以看到，第一个输出张量的轴-0长度（ 6 ）是两个输入张量轴-0长度的总和（ 3+3 ）； 第二个输出张量的轴-1长度（ 8 ）是两个输入张量轴-1长度的总和（ 4+4 ）。

有时，我们想通过逻辑运算符构建二元张量。 以X == Y为例： 对于每个位置，如果X和Y在该位置相等，则新张量中相应项的值为1。 这意味着逻辑语句X == Y在该位置处为真，否则该位置为0。

In [9]:
X == Y

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

In [11]:
# 对张量中的所有元素进行求和，会产生一个单元素张量。
X.sum(), Y.sum()

(tensor(66.), tensor(30.))

### 2.1.3. 广播机制
在上面的部分中，我们看到了如何在相同形状的两个张量上执行按元素操作。 在某些情况下，即使形状不同，我们仍然可以通过调用广播机制（broadcasting mechanism）来执行按元素操作。
这种机制的工作方式如下：
+ 首先，通过适当复制元素来扩展一个或两个数组， 以便在转换之后，两个张量具有相同的形状。
+ 其次，对生成的数组执行**按元素**操作。

In [12]:
a = torch.arange(3).reshape((3,1))
b = torch.arange(2).reshape((1,2))
a,b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

由于a和b分别是 3×1 和 1×2 矩阵，如果让它们相加，它们的形状不匹配。
我们将两个矩阵广播为一个更大的 3×2 矩阵，如下所示：矩阵a将复制列， 矩阵b将复制行，然后再按元素相加。

In [14]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

### 2.1.4. 索引和切片

X,X[-1], X[1:3]

In [21]:
X[0:2,:]=12
X

tensor([[12., 12., 12., 12.],
        [12., 12., 12., 12.],
        [ 8.,  9., 10., 11.]])

### 2.1.5. 节省内存
运行一些操作可能会导致为新结果分配内存。 例如，如果我们用Y = X + Y，我们将取消引用Y指向的张量，而是指向新分配的内存处的张量。

在下面的例子中，我们用Python的id()函数演示了这一点， 它给我们提供了内存中引用对象的确切地址。 运行Y = Y + X后，我们会发现id(Y)指向另一个位置。 这是因为Python首先计算Y + X，为结果分配新的内存，然后使Y指向内存中的这个新位置。

In [22]:
before = id(Y)
Y = Y + X
id(Y) == before

False

这可能是不可取的，原因有两个：首先，我们不想总是不必要地分配内存。 在机器学习中，我们可能有数百兆的参数，并且在一秒内多次更新所有参数。 通常情况下，我们希望原地执行这些更新。 其次，如果我们不原地更新，其他引用仍然会指向旧的内存位置， 这样我们的某些代码可能会无意中引用旧的参数。

幸运的是，执行原地操作非常简单。 我们可以使用切片表示法将操作的结果分配给先前分配的数组，例如Y[:] = <expression>。 为了说明这一点，我们首先创建一个新的矩阵Z，其形状与另一个Y相同， 使用zeros_like来分配一个全 0 的块

In [25]:
Z = torch.zeros_like(Y)
print('id(Z):',id(Z))
Z [:] = X + Y
print('id(Z):',id(Z))

id(Z): 2221760982288
id(Z): 2221760982288


### 2.1.6. 转换为其他Python对象

In [26]:
A = X.numpy()
B = torch.tensor(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)

In [28]:
# 要将大小为1的张量转换为Python标量，我们可以调用item函数或Python的内置函数。
a = torch.tensor([3.5])
a,a.item(),float(a),int(a)

(tensor([3.5000]), 3.5, 3.5, 3)